In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import preprocessing
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, Conv1D, GlobalMaxPool1D, concatenate

In [2]:
# train_data = pd.read_excel('감성대화/Training/최종-감정분류.xlsx')
# test_data = pd.read_excel('감성대화/Validation/val테스트.xlsx')

In [3]:
train_data = pd.read_excel('./archive/최종-감정분류.xlsx')
test_data = pd.read_excel('./archive/val테스트.xlsx')

In [4]:
train_data

,ages,document,label
0,청년,아내가 드디어 출산하게 되어서 정말 신이 나.,1
1,노년,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야.,0
2,청소년,고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워.,0
3,노년,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야. 너무 행복해.,1
4,노년,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어.,1
...,...,...,...
114284,노년,미리 미리 건강 챙기고 모두 안 아팠으면 좋겠어.,0
114285,노년,주변에 믿음직한 사람들에게서 정보도 많이 얻고 달콤한 말은 항상 의심하고 볼래.,0
114286,노년,친구들에게 내 마음을 터놓고 얘기하면 좀 나아질 것 같아.,0
114287,노년,남편에게 이런 내 마음을 솔직하게 얘기해 봐야겠어.,0


In [5]:
train_data.shape, test_data.shape

((114289, 3), (5130, 3))

In [6]:
features = train_data['document'].tolist()
labels = train_data['label'].tolist()
len(labels), len(features)

(114289, 114289)

In [100]:
from konlpy.tag import Mecab

mecab = Mecab(dicpath=r"C:/mecab/mecab-ko-dic")

#corpus = [mecab.morphs(text) for text in features]
#corpus

In [7]:
stopwords = ['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도',
            '를', '으로', '자', '에', '와', '한', '하다']

In [10]:
from tqdm import tqdm

In [12]:
import konlpy
from konlpy.tag import Okt
okt = Okt()

In [13]:
# OKT 정규화
# for sentence in tqdm(features):
#    temp_X = []
    # stem = True : 일정 수전의 정규화 수행 (이런 -> 이렇다, 만드는 -> 만들다)
    # 어근추출
#    temp_X = okt.morphs(sentence, stem=True)
#    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    # 한줄 반복문도 가능
    # [결과 for 반복문]
#    features.append(temp_X)

 50%|████████████████████████████████████▌                                    | 114289/228578 [08:05<08:05, 235.35it/s]


TypeError: No matching overloads found for kr.lucypark.okt.OktInterface.tokenize(list,java.lang.Boolean,java.lang.Boolean), options are:
	public java.util.List kr.lucypark.okt.OktInterface.tokenize(java.lang.String,java.lang.Boolean,java.lang.Boolean)



In [8]:
corpus = [preprocessing.text.text_to_word_sequence(text) for text in features]
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(corpus)
word_index = tokenizer.word_index
corpus

[['아내가', '드디어', '출산하게', '되어서', '정말', '신이', '나'],
 ['당뇨랑', '합병증', '때문에', '먹어야', '할', '약이', '열', '가지가', '넘어가니까', '스트레스야'],
 ['고등학교에', '올라오니', '중학교', '때보다', '수업이', '갑자기', '어려워져서', '당황스러워'],
 ['재취업이',
  '돼서',
  '받게',
  '된',
  '첫',
  '월급으로',
  '온',
  '가족이',
  '외식을',
  '할',
  '예정이야',
  '너무',
  '행복해'],
 ['빚을', '드디어', '다', '갚게', '되어서', '이제야', '안도감이', '들어'],
 ['이제',
  '곧',
  '은퇴할',
  '시기가',
  '되었어',
  '내가',
  '먼저',
  '은퇴를',
  '하고',
  '육',
  '개월',
  '후에',
  '남편도',
  '은퇴를',
  '하고',
  '그러면',
  '둘이',
  '하루',
  '종일',
  '함께',
  '있겠네',
  '생각만',
  '해도',
  '숨이',
  '막혀'],
 ['사십', '대에', '접어들면서', '머리카락이', '많이', '빠져', '고민이야'],
 ['이제', '돈이라면', '지긋지긋해'],
 ['친구들이', '나를', '괴롭혀', '부모님과', '선생님께', '얘기했는데도', '믿어주지', '않아'],
 ['친구', '때문에', '눈물', '나'],
 ['새로',
  '간',
  '치과에서',
  '조금',
  '썩기만',
  '한',
  '이를',
  '임플란트를',
  '하라고',
  '했어',
  '과잉진료는',
  '안',
  '한다더니'],
 ['오늘',
  '선생님이',
  '나보고',
  '앞에',
  '나와',
  '문제를',
  '풀어보라고',
  '했는데',
  '답을',
  '외워서',
  '문제를',
  '풀었어'],
 ['내가', '무능해서', '자기가', '좋은', '대학에', '못', '들어갔다는'

In [102]:
tokenizer

In [103]:
sequences = tokenizer.texts_to_sequences(corpus)

In [104]:
sequences

[[82, 6, 554, 582, 5, 21, 47, 61, 75, 404, 2, 10, 1],
 [649,
  134,
  1830,
  99,
  9,
  89,
  214,
  37,
  392,
  2,
  637,
  338,
  6,
  1540,
  107,
  201,
  26,
  1],
 [919, 9, 3558, 72, 2116, 81, 142, 773, 2, 221, 2817, 173, 112, 1],
 [2325,
  2,
  339,
  54,
  21,
  199,
  1010,
  710,
  48,
  606,
  117,
  2,
  1213,
  4,
  37,
  1358,
  2,
  26,
  1,
  27,
  309,
  15,
  1],
 [863, 4, 554, 49, 552, 21, 47, 61, 1656, 2415, 2, 13, 3, 1],
 [104,
  523,
  220,
  37,
  1018,
  6,
  47,
  40,
  3,
  1,
  16,
  6,
  213,
  220,
  11,
  5,
  8,
  1090,
  1493,
  289,
  9,
  92,
  12,
  220,
  11,
  5,
  8,
  1,
  1154,
  446,
  2,
  493,
  1200,
  256,
  18,
  22,
  56,
  1,
  50,
  42,
  210,
  1329,
  2,
  3291,
  1],
 [141, 303, 197, 9, 6143, 102, 3657, 2, 79, 1898, 156, 2, 26, 1],
 [104, 70, 2, 885, 1848, 15, 1],
 [24,
  13,
  2,
  10,
  11,
  2045,
  1,
  120,
  60,
  69,
  172,
  60,
  190,
  118,
  28,
  19,
  12,
  209,
  3,
  55,
  17,
  32,
  14,
  1],
 [24, 99, 9, 342, 10, 1

In [105]:
MAX_SEQ_LEN = 100
padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post')

In [106]:
ds = tf.data.Dataset.from_tensor_slices((padded_seqs, labels))
ds = ds.shuffle(len(features))

In [107]:
train_size = int(len(padded_seqs)*0.7)
val_size = int(len(padded_seqs)*0.2)
test_size = int(len(padded_seqs)*0.1)

In [108]:
train_ds = ds.take(train_size).batch(20)
val_ds = ds.skip(train_size).take(val_size).batch(20)
test_ds = ds.skip(train_size+val_size).take(test_size).batch(20)

In [109]:
dropout_ptob = 0.5
EMB_SIZE = 128
EPOCH = 10
VOCAB_SIZE = len(word_index)+1

In [110]:
input_layer = Input(shape=(MAX_SEQ_LEN,))
embeding_layer = Embedding(VOCAB_SIZE, EMB_SIZE, input_length=MAX_SEQ_LEN)(input_layer)
dropout_emb = Dropout(rate=dropout_ptob)(embeding_layer)

In [111]:
conv1 = Conv1D(
    filters=128,
    kernel_size=3,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb)
pool1=GlobalMaxPool1D()(conv1)

In [112]:
conv2 = Conv1D(
    filters=128,
    kernel_size=4,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb)
pool2=GlobalMaxPool1D()(conv2)

In [113]:
conv3 = Conv1D(
    filters=128,
    kernel_size=5,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb)
pool3=GlobalMaxPool1D()(conv3)

In [114]:
concat=concatenate([pool1, pool2, pool3])

In [115]:
hidden = Dense(128, activation=tf.nn.relu)(concat)
dropout_hidden = Dropout(rate=dropout_ptob)(hidden)
logits = Dense(3, name='logits')(dropout_hidden)
predictions = Dense(4, activation=tf.nn.softmax)(logits)

In [116]:
model = Model(inputs = input_layer, outputs=predictions)
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [117]:
model.fit(train_ds, validation_data=val_ds, epochs=EPOCH, verbose=1)

Epoch 1/10
4001/4001 [==============================] - 88s 22ms/step - loss: 1.0638 - accuracy: 0.5279 - val_loss: 0.9209 - val_accuracy: 0.612214 - loss: 1.3374 - accura - ETA: 1:14 - loss: 1.3348 - accuracy:  - ETA: 1:14 - loss: 1.3316 - accuracy: 0.34 - ETA: 1:14 - loss: 1.3320 - accuracy - ETA: 1:13 - loss: 1.3314 - ac - ETA: 1:13 - loss: 1.3265 - accu - ETA: 1:13 - loss: 1.3244 - accura - ETA: 1:12 - loss: 1.3225 - accuracy: 0. - ETA: 1:12 - loss: 1.3217 - accuracy: 0. - ETA: 1:12 - loss: 1.3209 - accuracy - ETA: 1:12 - l - ETA: 1:12 - loss: 1.3108 -  - ETA: 1:11 - loss: 1.3054 - accu - ETA: 1:11 - loss: 1.3024 - ac - ETA: 1:10 - loss: 1 - ETA: 1:10 - l - ETA: 1:09 - loss: 1.2767 -  - ETA: 1:08 - loss: 1.2712  - ETA: 1:06 - loss: - ETA: 1:06 - loss: - ETA: 1:05 - loss: 1.2383 - accu - ETA - ETA: 1:04 - loss: 1.2273 - accuracy: 0.42 - ETA: 1:04 - loss: 1.2264 - accuracy: 0. - ETA: 1:03 - loss: 1.2 - E - ETA: 1:02 - loss: 1.2127 - accuracy - ETA: 1:01 - loss: 1.2108  - ETA: 59s - l

In [118]:
loss, accuracy = model.evaluate(test_ds, verbose=1)
print('Accuracy : %f' %(accuracy*100))
print('loss : %f' % (loss))

572/572 [==============================] - 2s 2ms/step - loss: 0.6461 - accuracy: 0.7486
Accuracy : 74.859995
loss : 0.646119


In [119]:
model.save('cnn_model_mecab_{}.h5'.format(accuracy))

### 모델 사용하기

In [120]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import preprocessing
from tensorflow.keras.models import Model, load_model

In [121]:
train_data = pd.read_excel('./archive/최종-감정분류.xlsx')
test_data = pd.read_excel('./archive/val테스트.xlsx')

In [ ]:
stopwords = ['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도',
            '를', '으로', '자', '에', '와', '한', '하다']

In [122]:
features = test_data['document'].tolist()
labels = test_data['label'].tolist()

In [123]:
sequences[3]

[2325,
 2,
 339,
 54,
 21,
 199,
 1010,
 710,
 48,
 606,
 117,
 2,
 1213,
 4,
 37,
 1358,
 2,
 26,
 1,
 27,
 309,
 15,
 1]

In [124]:
#corpus = [preprocessing.text.text_to_word_sequence(text) for text in features]
tokenizer = preprocessing.text.Tokenizer() # 토크나이저 객체 생성
tokenizer.fit_on_texts(corpus)
sequences = tokenizer.texts_to_sequences(corpus)
corpus

[['요즘', '부모님과', '많이', '부딪혀'],
 ['엄마가', '결국', '집을', '나갔어', '너무', '너무', '슬퍼'],
 ['학교에서', '한', '친구를', '괴롭히는', '무리에게', '그만하라고', '했어'],
 ['이번에',
  '팀장님이',
  '간단한',
  '조사',
  '업무를',
  '부탁하셨는데',
  '내가',
  '잘못',
  '처리했어',
  '너무',
  '절망적이야'],
 ['남편이', '이혼할', '때', '위자료를', '주지', '않으려고', '변호사를', '고용했어'],
 ['친구들과',
  '노후에',
  '대한',
  '이야기를',
  '하다',
  '보니',
  '서로',
  '노후',
  '자금',
  '차이가',
  '상당히',
  '컸어'],
 ['직장에서', '모함을', '받았어', '난', '정말', '억울해'],
 ['요즘', '딸에게', '뭔가', '물어보면', '신경질부터', '내는데', '화가', '나'],
 ['언제까지', '대출금을', '갚으며', '살아야', '할까', '포기하고', '싶어지고', '너무', '슬퍼'],
 ['이', '업무는', '하루', '만에', '끝낼', '수', '있을', '것', '같아'],
 ['나',
  '대학에',
  '쉽게',
  '갈',
  '수',
  '있을',
  '것',
  '같아',
  '학원',
  '선생님들',
  '실력이',
  '정말',
  '믿을',
  '만해'],
 ['작년에', '왔던', '이직', '제안을', '거절한', '게', '후회돼'],
 ['나이는', '먹어', '가는데', '경제적인', '여유가', '없어서', '큰일이네'],
 ['이번에', '약이', '바뀌었는데', '정말', '구역질이', '나'],
 ['내', '친구가', '따돌림을', '당하고', '있어'],
 ['엄마는', '나를', '걱정해서', '충고를', '했는데', '나는', '짜증만', '냈어'],
 ['심혈을', '기울여', '작성한', '기획안이'

In [125]:
MAX_SEQ_LEN = 100
padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post')

In [126]:
ds = tf.data.Dataset.from_tensor_slices((padded_seqs, labels))
ds = ds.shuffle(len(features))
test_ds = ds.take(2000).batch(20)

In [132]:
model = load_model('cnn_model_mecab_0.7485999464988708.h5')
model.summary()
model.evaluate(test_ds, verbose=2)

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 100, 128)     2332800     input_3[0][0]                    
__________________________________________________________________________________________________
dropout_4 (Dropout)             (None, 100, 128)     0           embedding_2[0][0]                
__________________________________________________________________________________________________
conv1d_6 (Conv1D)               (None, 98, 128)      49280       dropout_4[0][0]                  
____________________________________________________________________________________________

[1.9900617599487305, 0.26899999380111694]

In [133]:
print('단어 시퀀스 : ', corpus[13])
print('단어 인덱스 시퀀스 : ', padded_seqs[13])
print('문장 분류(정답) : ', labels[13])

단어 시퀀스 :  ['이번에', '약이', '바뀌었는데', '정말', '구역질이', '나']
단어 인덱스 시퀀스 :  [  31 1910 2690   13  550    4    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
문장 분류(정답) :  2


In [134]:
picks = [1195]
predict = model.predict(padded_seqs[picks])
predict_class = tf.math.argmax(predict, axis=1)
print('감정 예측 점수 : ', predict)
print('감정 예측 클래스 : ', predict_class.numpy())

감정 예측 점수 :  [[7.3865992e-03 6.5472013e-05 7.4377959e-03 9.8511016e-01]]
감정 예측 클래스 :  [3]


In [130]:
text = '이번에 약이 바뀌었는데 정말 구역질이 나'

In [131]:
corpus = [preprocessing.text.text_to_word_sequence(text)

SyntaxError: unexpected EOF while parsing (<ipython-input-131-b9e71bf883d2>, line 1)

In [ ]:
print(picks)

In [ ]:
padded_seqs[picks]